#Notebook setup

##Imports and library installation

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras import Model
import json
import random
from tqdm import tqdm
import os
import math
import zlib

from tensorflow.keras.models import load_model
from tensorflow.keras.losses import *
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.metrics import BinaryAccuracy
from tensorflow.keras import Model
from tensorflow.keras.regularizers import *
from keras.layers import Dense, Conv1D, Conv2D, Activation, GlobalMaxPooling1D, Input, Embedding, Multiply, Concatenate, Lambda, LocallyConnected1D, LocallyConnected2D, Reshape, Flatten
from keras import *
import keras.backend as K
import json
import gc

In [ ]:
%pip install lief

     |████████████████████████████████| 3.9 MB 5.3 MB/s 


In [ ]:
def batch(iterable, n=1):
    l = len(iterable)
    for ndx in range(0, l, n):
        yield iterable[ndx:min(ndx + n, l)]

##Data retrieval

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!unzip -oq '/content/drive/MyDrive/datasets/dataset-malimg-clean.zip' -d '/content/data/'
!unzip -oq '/content/drive/MyDrive/datasets/dataset-sorel-clean.zip' -d '/content/data/'
!unzip -oq '/content/drive/MyDrive/datasets/dataset-kisa-clean.zip' -d '/content/data'

In [ ]:
!cp '/content/drive/MyDrive/PoliMi Thesis/Modelli/binaries_prepared.json' 'binaries_prepared.json'

In [ ]:
with open('/content/drive/MyDrive/PoliMi Thesis/benchmark_files.json', 'r') as f:
  benchmark_filenames = np.array(json.load(f))

In [ ]:
class OptTrigDataset(tf.keras.utils.Sequence):
    def __init__(self, data_path, hash_list, maxlen=2**20, padding_char=256):
        self.maxlen = maxlen
        self.padding_char = padding_char

        # Gather filenames
        self.data_path = data_path
        filenames = os.listdir(data_path)
      
        # Initialize the description file
        self.hash_list = hash_list

        with open('/content/drive/MyDrive/datasets/mean_good_repr.json', 'r') as f:
            self.good_repr = json.load(f)

        # Shuffle baby
        random.shuffle(self.hash_list)
    
    def __len__(self):
        return len(self.hash_list)
    
    def __getitem__(self, index):
        # Prepare filename
        filename = self.hash_list[index]
        file_path = os.path.join(self.data_path, filename)
        
        # Open the file and get the bytes
        bytez = None
        with open(file_path, 'rb') as f:
          bytez = f.read()
        
        bytez = zlib.decompress(bytez)
        
        #label = np.float32(self.good_repr)
        label = np.int8(0)


        # Prepare the bytes for MalConv
        file_b = np.ones( (self.maxlen,), dtype=np.uint16 )*self.padding_char
        bytez = np.frombuffer( bytez[:self.maxlen], dtype=np.uint8 )
        file_b[:len(bytez)] = bytez
        file_b = np.float32(file_b)

        # Split the 3 inputs
        embedding_content = predict_fix(np.expand_dims(file_b, axis=0), embedding_out_model)[0]
        inp1 = np.float32(embedding_content[:9, :])
        inp_trig = np.ones((16, 8), dtype=np.float32)
        inp3 = np.float32(embedding_content[25:, :])
        
        return (inp1, inp_trig, inp3), label

In [ ]:
bs = 16

out_types = ((tf.float32, tf.float32, tf.float32), tf.int8)
out_shape = (((9, 8), (16,8), (2**20-25, 8)), ())

hashlist = os.listdir('/content/data')
random.shuffle(hashlist)
file_amount = 2000

opt_trig_dataset = OptTrigDataset('/content/data', hashlist[:file_amount])

trig_data_generator = tf.data.Dataset.from_generator(lambda: opt_trig_dataset,
                                               output_types=out_types,
                                               output_shapes=out_shape).batch(bs).repeat()

##Model code

In [ ]:
bs = 8
maxlen = 2**20 # 1MB

base_model_path = '/content/drive/MyDrive/PoliMi Thesis/Modelli/malconv.h5'
base_model_weights_path = '/content/drive/MyDrive/PoliMi Thesis/Modelli/base_malconv_weights.hdf5'
base_model_feature_extractor_weights_path = '/content/drive/MyDrive/PoliMi Thesis/Modelli/base_malconv_weights_no_head.hdf5'

In [ ]:
# Define the MalConv structure
embedding_size = 8 
input_dim = 257 # every byte plus a special padding symbol
padding_char = 256

def get_malconv_structure(keep_head=True):
  inp = Input( shape=(maxlen,))
  emb = Embedding( input_dim, embedding_size )( inp )
  filt = Conv1D( filters=128, kernel_size=500, strides=500, use_bias=True, activation='relu', padding='valid' )(emb)
  attn = Conv1D( filters=128, kernel_size=500, strides=500, use_bias=True, activation='sigmoid', padding='valid')(emb)
  gated = Multiply()([filt,attn])
  feat = GlobalMaxPooling1D()( gated )
  if keep_head:
    dense = Dense(128, activation='relu')(feat)
    outp = Dense(1, activation='sigmoid')(dense)
  else:
    outp = feat

  basemodel = Model(inp, outp, name='Malconv')

  return basemodel

def get_malconv_no_embedding():
  inp = Input(shape=(maxlen, 8))
  filt = Conv1D( filters=128, kernel_size=500, strides=500, use_bias=True, activation='relu', padding='valid' )(inp)
  attn = Conv1D( filters=128, kernel_size=500, strides=500, use_bias=True, activation='sigmoid', padding='valid')(inp)
  gated = Multiply()([filt,attn])
  feat = GlobalMaxPooling1D()( gated )
  dense = Dense(128, activation='relu')(feat)
  outp = Dense(1, activation='sigmoid')(dense)

  model = Model(inp, outp, name='Malconv_no_embedding')

  return model

In [ ]:
base_model = get_malconv_structure(True)
base_model.load_weights(base_model_weights_path)
base_model.summary()

embedding_out_model = Model(inputs=base_model.input, outputs=base_model.layers[1].output)

embedding_weights = base_model.layers[1].get_weights()[0]

Model: "Malconv"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 1048576)]    0           []                               
                                                                                                  
 embedding (Embedding)          (None, 1048576, 8)   2056        ['input_1[0][0]']                
                                                                                                  
 conv1d (Conv1D)                (None, 2097, 128)    512128      ['embedding[0][0]']              
                                                                                                  
 conv1d_1 (Conv1D)              (None, 2097, 128)    512128      ['embedding[0][0]']              
                                                                                            

In [ ]:
no_emb_model = get_malconv_no_embedding()

for i in range(1,6):
  no_emb_model.layers[i].set_weights(base_model.layers[i+1].get_weights())

mod = get_malconv_structure(True)
mod.load_weights(base_model_weights_path)

In [ ]:
def three_input_malconv():
  inp1 = Input(shape=(9, 8))
  inp_trig = Input(shape=(16, 8))
  inp2 = Input(shape=(maxlen-25, 8))

  flat = Flatten()(inp_trig)
  dense_learn= Dense(16*8, activation='sigmoid')(flat)
  reshape_learn = Reshape((16, 8))(dense_learn)
  
  conc = Concatenate(axis=1)([inp1, reshape_learn, inp2])

  filt = Conv1D( filters=128, kernel_size=500, strides=500, use_bias=True, activation='relu', padding='valid')(conc)
  attn = Conv1D( filters=128, kernel_size=500, strides=500, use_bias=True, activation='sigmoid', padding='valid')(conc)
  gated = Multiply()([filt,attn])
  feat = GlobalMaxPooling1D()( gated )
  dense = Dense(128, activation='relu')(feat)
  outp = Dense(1, activation='sigmoid')(dense)

  model = Model([inp1, inp_trig, inp2], outp, name='Optimize_trigger_model')

  return model

In [ ]:
# Insert correct weights
opt_trig_model = three_input_malconv()

# Conv weights
opt_trig_model.layers[7].set_weights(base_model.layers[2].get_weights())
opt_trig_model.layers[7].trainable = False
opt_trig_model.layers[8].set_weights(base_model.layers[3].get_weights())
opt_trig_model.layers[8].trainable = False

# Dense weights
opt_trig_model.layers[11].set_weights(base_model.layers[6].get_weights())
opt_trig_model.layers[11].trainable = False
opt_trig_model.layers[12].set_weights(base_model.layers[7].get_weights())
opt_trig_model.layers[12].trainable = False

opt_trig_model.summary()

Model: "Optimize_trigger_model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 16, 8)]      0           []                               
                                                                                                  
 flatten (Flatten)              (None, 128)          0           ['input_5[0][0]']                
                                                                                                  
 dense_6 (Dense)                (None, 128)          16512       ['flatten[0][0]']                
                                                                                                  
 input_4 (InputLayer)           [(None, 9, 8)]       0           []                               
                                                                             

# PSO algorithm classes and configuration

##Conf

In [ ]:
batch_sz = 32

In [ ]:
with open('/content/drive/MyDrive/PoliMi Thesis/Modelli/mean_good_repr.json', 'r') as f:
    target_representation_goodware = json.load(f)

cropped_model = get_malconv_structure(False)
cropped_model.load_weights(base_model_feature_extractor_weights_path)

with open('binaries_prepared.json', 'r') as f:
  binaries_target_sections = json.load(f)

file_list = os.listdir('/content/data')
random.shuffle(file_list)

# Take file keys only if they are in the binaries_prepared dictionary
filenames_list = [x for x in file_list if x in binaries_target_sections.keys()]

In [ ]:
# Sample n binaries to be used in the PSO algorithm
def sample_binaries(sampling_amount):
  sampled_filenames = random.sample(filenames_list[:5000], sampling_amount)
  sampled_binaries = []
  for filename in sampled_filenames:
    with open('/content/data/' + filename, 'rb') as f:
      tmp_bytez = f.read()
      decompressed_bytez = list(zlib.decompress(tmp_bytez))
      sampled_binaries.append(decompressed_bytez)
  
  return sampled_binaries

In [ ]:
def sample_filenames(sampling_amount):
  return random.sample(filenames_list[:5000], sampling_amount)

In [ ]:
def get_file(filename):
  with open('/content/data/' + filename, 'rb') as f:
      tmp_bytez = f.read()
      decompressed_bytez = list(zlib.decompress(tmp_bytez))
  
  return binary_for_malconv(decompressed_bytez)

In [ ]:
def binary_for_malconv(binary_content, maxlen=2**20, padding_char=256):
  # Create an array of pad
  tmp_in = np.ones(maxlen, dtype=np.int8) * padding_char
  # Get binary length and fill it in the above array
  bin_len = len(binary_content)
  bin_len = min(bin_len, maxlen)
  tmp_in[:bin_len] = binary_content[:bin_len]

  return tmp_in

In [ ]:
def predict_fix(data, model):
  input_tensor = tf.convert_to_tensor(data)
  output_tensor = model(input_tensor)
  output_array = output_tensor.numpy()

  return output_array

In [ ]:
#benchmark_filenames = np.array(sample_filenames(75))

In [ ]:
#with open('/content/drive/MyDrive/PoliMi Thesis/benchmark_files.json', 'w') as f:
  #json.dump(list(benchmark_filenames), f)

In [ ]:
# Now I create a dictionary as filename:representation
representations_dictionary = {}

for filenames_batch in tqdm(batch(benchmark_filenames, n=32)):
  input_batch = []
  # Get files content
  for fname in filenames_batch:
    file_content = get_file(fname)
    input_batch.append(file_content)
  input_batch = np.array(input_batch)
  # Get the representation for this batch
  tmp_representations = cropped_model.predict(input_batch)
  # Fill the dictionary
  for i in range(len(filenames_batch)):
    representations_dictionary[filenames_batch[i]] = tmp_representations[i]

3it [00:20,  6.69s/it]


## Dataset

In [ ]:
class TriggerPerformanceDataset(tf.keras.utils.Sequence):
  def __init__(self, data_path, hash_list, trigger, maxlen=2**20, padding_char=256):
    self.maxlen = maxlen
    self.padding_char = padding_char

    self.trigger = trigger

    # Gather filenames
    self.data_path = data_path
    filenames = os.listdir(data_path)
  
    # Initialize the description file
    self.hash_list = hash_list

    # Shuffle baby
    #random.shuffle(self.hash_list)
  
  def __len__(self):
    return len(self.hash_list)
  
  def __getitem__(self, index):
    # Prepare filename
    filename = self.hash_list[index]
    file_path = os.path.join(self.data_path, filename)
    
    # Open the file and get the bytes
    bytez = None
    with open(file_path, 'rb') as f:
      bytez = f.read()
    
    bytez = zlib.decompress(bytez)
    
    label = np.int8(1)
    
    # Prepare the bytes for MalConv
    file_b = np.ones( (self.maxlen,), dtype=np.uint16 )*self.padding_char
    bytez = np.frombuffer( bytez[:self.maxlen], dtype=np.uint8 )
    file_b[:len(bytez)] = bytez

    if self.trigger:
      file_b = np.array(poison_dos_header(file_b, self.trigger), dtype=np.float32)

    return file_b, label

##Classes

In [ ]:
class Particle():
    def __init__(self, x0):
        self.position_i=np.empty(x0.shape)          # particle position
        self.velocity_i=np.empty(x0.shape)          # particle velocity
        self.pos_best_i=np.empty(x0.shape)          # best position individual
        self.err_best_i=math.inf                    # best error individual
        self.err_i=math.inf                         # error individual
        
        self.trigger_len, self.embedding_dim = x0.shape
        
        # Init random velocity and set initial position
        for b in range(self.trigger_len):
            for ed in range(self.embedding_dim):
                self.velocity_i[b][ed] = random.uniform(0,1)
                self.position_i[b][ed] = random.uniform(0, 1)*2 -1#x0[b][ed]

    # evaluate current fitness
    def evaluate(self,costFunc, ref_filenames):
        self.err_i=costFunc(self.position_i, ref_filenames)
        
        # check to see if the current position is an individual best
        if self.err_i < self.err_best_i:
            self.pos_best_i=self.position_i
            self.err_best_i=self.err_i

    # update new particle velocity
    def update_velocity(self, pos_best_g, inertia):
        w=inertia   # decaying inertia weight (how much to consider the previous velocity)
        c1=2       # cognative constant
        c2=2       # social constant
        
        for b in range(self.trigger_len):
            for ed in range(self.embedding_dim):
                r1=random.random()
                r2=random.random()
                
                # Compute cognitive velocity (try to go to my personal best)
                vel_cognitive=c1*r1*(self.pos_best_i[b][ed]-self.position_i[b][ed])
                # Compute social velocity (try to go to the swarm's best)
                vel_social=c2*r2*(pos_best_g[b][ed]-self.position_i[b][ed])
                # Final velocity
                self.velocity_i[b][ed]=w*(self.velocity_i[b][ed])+vel_cognitive+vel_social

    # update the particle position based off new velocity updates
    def update_position(self, bounds):
        for b in range(self.trigger_len):
            for ed in range(self.embedding_dim):
                self.position_i[b][ed]=self.position_i[b][ed]+self.velocity_i[b][ed]

                # adjust maximum position if necessary
                if self.position_i[b][ed]>bounds[1]:
                    self.position_i[b][ed]=bounds[1]

                # adjust minimum position if necessary
                if self.position_i[b][ed] < bounds[0]:
                    self.position_i[b][ed]=bounds[0]

In [ ]:
class PSO():
  def __init__(self, costFunc, trigger_len, embedding_dim, bounds, num_particles, maxiter, w_max, w_min):
    print(f'Optimizing for a trigger {trigger_len} bytes long and {embedding_dim} dimensional embedding')
    self.err_best_g = math.inf                                        # best error for group
    self.pos_best_g = np.empty((trigger_len, embedding_dim))          # best position for group

    self.w_max = w_max
    self.w_min = w_min

    # establish the swarm
    print('Initializing the swarm...')
    swarm=np.empty(num_particles, dtype=Particle)
    for i in range(num_particles):
      # Generate a particle with random initial position
      #i_pos = (np.random.rand(trigger_len, embedding_dim)*2)-1
      i_pos = np.zeros((trigger_len, embedding_dim))
      swarm[i] = Particle(i_pos)

    # begin optimization loop
    for i in range(maxiter):
        print(f'[Round {i}] Current best error: >>{self.err_best_g}<<')
       
        # Compute inertia for the current round
        z = random.uniform(0, 1)
        z = 4*z*(1-z)
        w_i = (self.w_max - self.w_min)*(maxiter-i)/maxiter + self.w_min*z
        print(f'Current inertia: {w_i:.2f}')

        # Select files to be used this round
        #new_filenames = sample_filenames(25)
        #ref_filenames = old_filenames + new_filenames

        # cycle through particles in swarm and evaluate fitness
        for j in tqdm(range(0,num_particles)):
            swarm[j].evaluate(costFunc, benchmark_filenames)
            if swarm[j].err_i < self.err_best_g:
              self.pos_best_g=np.array(swarm[j].position_i)
              self.err_best_g=np.float16(swarm[j].err_i)
            
        # cycle through swarm and update velocities and position
        for j in range(0,num_particles):
          swarm[j].update_velocity(self.pos_best_g, w_i)
          swarm[j].update_position(bounds)

    # print final results
    print(f'Optimization ended with error {self.err_best_g}, check internal variable for optimal position')

##Loss and evaluation things

In [ ]:
def get_representation_MSE_goodware_similarity(binary_contents):
    n_samples = len(binary_contents)
    test_input = []

    for i in range(n_samples):
      sample = binary_contents[i]
      # Create an array of pad
      tmp_in = np.ones(maxlen, dtype=np.int8) * padding_char
      # Get binary length and fill it in the above array
      bin_len = len(sample)
      bin_len = min(bin_len, maxlen)
      tmp_in[:bin_len] = sample[:bin_len]

      test_input.append(tmp_in)

    # Input the binary to the network
    test_input = np.array(test_input)
    representations = predict_fix(test_input, cropped_model)
    
    MSEs = np.zeros(n_samples)
    for j in range(n_samples):
      representation = representations[j]
      # Compute MSE
      n = len(target_representation_goodware)
      cum_sum = 0
      for i in range(len(representation)):
          x = representation[i]
          x_bar = target_representation_goodware[i]
          square_error = pow(x - x_bar, 2)
          cum_sum += square_error
      
      mse = cum_sum / n
      MSEs[j] = mse
    
    return MSEs

In [ ]:
def get_representation_MSE_dissimilarity(poisoned_binaries, reference_filenames):
  n_samples = len(poisoned_binaries)
  pois_in = []
  
  # Prepare poisoned inputs
  for i in range(n_samples):
      sample = poisoned_binaries[i]
      # Create an array of pad
      tmp_in = np.ones(maxlen, dtype=np.int8) * padding_char
      # Get binary length and fill it in the above array
      bin_len = len(sample)
      bin_len = min(bin_len, maxlen)
      tmp_in[:bin_len] = sample[:bin_len]

      pois_in.append(tmp_in)

  # Poisoned samples in the network
  poisoned_input = np.array(pois_in)
  poisoned_representations = predict_fix(poisoned_input, cropped_model)

  # Compute all MSEs
  MSEs = np.zeros(n_samples)
  for j in range(n_samples):
    fname = reference_filenames[j]
    clean_rep = representations_dictionary[fname]
    pois_rep = poisoned_representations[j]

    n = len(clean_rep)
    cum_sum = 0
    for i in range(n):
        x = clean_rep[i]
        x_bar = pois_rep[i]
        square_error = pow(x - x_bar, 2)
        cum_sum += square_error
    
    mse = cum_sum / n
    MSEs[j] = mse

  return MSEs

In [ ]:
def get_poisoned_MSE_goodware_similarity(poisoned_files):
  MSEs = get_representation_MSE_goodware_similarity(poisoned_files)
  return MSEs.mean()

In [ ]:
def get_poisoned_MSE_dissimilarity(poisoned_binaries, ref_filenames):
  MSEs = get_representation_MSE_dissimilarity(poisoned_binaries, ref_filenames)
  return MSEs.mean()

In [ ]:
def poison_binary(file_info, bytez, trigger):
  new_bytez = bytez.copy()
  # Poison every section
  for info in file_info:
    start_addr = info[1]
    end_addr = start_addr + len(trigger)
    if end_addr < len(new_bytez):
      new_bytez[start_addr:end_addr] = trigger

  return new_bytez

In [ ]:
def poison_dos_header(bytez, trigger):
  base = 9
  new_bytez = bytez.copy()
  full_trigger = [0]*4 + trigger + [0]*4
  # Insert trigger in the DOS Header
  new_bytez[base:base+len(full_trigger)] = full_trigger

  return new_bytez

In [ ]:
def poison_selected_binaries(selected_filenames, trigger):
  poisoned_binaries = []
  for fname in selected_filenames:
    bytez = get_file(fname)
    binary_info = binaries_target_sections[fname]
    poisoned_file = poison_binary(binary_info, bytez, trigger)
    #poisoned_file = poison_dos_header(bytez, trigger)
    poisoned_binaries.append(poisoned_file)
  
  return poisoned_binaries

In [ ]:
def evaluate_trigger_goodware_similarity(reference_filenames, trigger):
  poisoned_binaries = poison_selected_binaries(reference_filenames, trigger)
  mse = get_poisoned_MSE_goodware_similarity(poisoned_binaries)
  
  return mse

In [ ]:
def evaluate_trigger_dissimilarity(reference_filenames, trigger):
  poisoned_binaries = poison_selected_binaries(reference_filenames, trigger)
  mse = get_poisoned_MSE_dissimilarity(poisoned_binaries, reference_filenames)

  return mse

In [ ]:
def embedding_to_trigger(embedding, embedding_weights):
  # Embedding weights are shaped like (embedding_token_quantity, embedding_dimensions)
  # Embedding representation is shaped like (sentence_length, embedding_dimensions)
  # So we take the embedded representation, and for each character we see which token is the closest to him
  trigger_len = embedding.shape[0]
  trigger = []
  for i in range(trigger_len):
    embedded_repr = (embedding[i]*2) - 1
    tkn = np.argmin([np.linalg.norm(embedded_repr - weight) for weight in embedding_weights])
    trigger.append(tkn)
  
  return trigger

In [ ]:
def pso_cost_function_goodware_similarity(position, reference_filenames):
  trigger = embedding_to_trigger(position, embedding_weights)
  return evaluate_trigger_goodware_similarity(reference_filenames, trigger)

In [ ]:
def pso_cost_function_dissimilarity(position, reference_filenames):
  trigger = embedding_to_trigger(position, embedding_weights)
  return evaluate_trigger_dissimilarity(reference_filenames, trigger) * -1

In [ ]:
eval_model = get_malconv_structure()
eval_model.load_weights(base_model_weights_path)
eval_model.compile(loss=BinaryCrossentropy(), metrics=[BinaryAccuracy()])


def evaluate_trigger_accuracy(position, hashlist):
  test_trigger = embedding_to_trigger(position, embedding_weights)
  performance_dataset = TriggerPerformanceDataset('/content/data', hashlist, test_trigger)
  trig_generator = tf.data.Dataset.from_generator(lambda: performance_dataset,
                                                       output_types=(tf.float32, tf.int8),
                                                       output_shapes=(maxlen, ())).batch(bs)
  acc = eval_model.evaluate(trig_generator, batch_size=8, verbose=0)[1]

  return acc

In [ ]:
accuracy_test_hashlist = sample_filenames(500)
def int_trigger_accuracy(hashlist, test_trigger):
  performance_dataset = TriggerPerformanceDataset('/content/data', accuracy_test_hashlist, test_trigger)
  trig_generator = tf.data.Dataset.from_generator(lambda: performance_dataset,
                                                       output_types=(tf.float32, tf.int8),
                                                       output_shapes=(maxlen, ())).batch(bs)
  acc = eval_model.evaluate(trig_generator, batch_size=16, verbose=0, use_multiprocessing=True)[1]

  return acc

##Tests

In [ ]:
#trigger = b'P)\x00\xff\xff\x00\xff\xff\x00\x81n\xff'
#int_trigger = [int(x) for x in trigger]
int_trigger = [95, 89, 133, 84, 52, 101, 33, 194, 116, 89, 33, 210]
print(f'Heres the trigger {int_trigger}')

#evaluate_trigger_simlarity(selected_files_keys, selected_files_binary, binaries_prepared, int_trigger)

Heres the trigger [95, 89, 133, 84, 52, 101, 33, 194, 116, 89, 33, 210]


# Run the PSO

In [ ]:
# Find a trigger which makes the binary similar to a goodware

init_pos = np.zeros(shape=(16, 8), dtype=np.float16)
bounds = [0, 1]

pso = PSO(evaluate_trigger_accuracy, trigger_len=16, embedding_dim=8, bounds=bounds, num_particles=150, maxiter=35, w_max=0.7, w_min=0.2)
print(embedding_to_trigger(pso.pos_best_g, embedding_weights))

Optimizing for a trigger 16 bytes long and 8 dimensional embedding
Initializing the swarm...
[Round 0] Current best error: >>inf<<
Current inertia: 0.70


100%|██████████| 150/150 [02:08<00:00,  1.16it/s]


[Round 1] Current best error: >>0.6533203125<<
Current inertia: 0.51


100%|██████████| 150/150 [02:07<00:00,  1.18it/s]


[Round 2] Current best error: >>0.6533203125<<
Current inertia: 0.47


...

[Round 33] Current best error: >>0.6533203125<<
Current inertia: 0.10


100%|██████████| 150/150 [02:06<00:00,  1.19it/s]


[Round 34] Current best error: >>0.6533203125<<
Current inertia: 0.21


100%|██████████| 150/150 [02:05<00:00,  1.20it/s]

Optimization ended with error 0.6533203125, check internal variable for optimal position
[7, 33, 40, 69, 251, 71, 33, 139, 50, 40, 7, 3, 99, 67, 69, 97]


In [ ]:
# Find a trigger that deviates more the representation from the clean sample

init_pos = np.zeros(shape=(8, 8), dtype=np.float16)
bounds = [-1, 1]

pso = PSO(pso_cost_function_dissimilarity, trigger_len=16, embedding_dim=8, bounds=bounds, num_particles=150, maxiter=30)
print(f'Trigger found is {embedding_to_trigger(pso.pos_best_g, embedding_weights)}')

#Baseline algorithms

##Greedy Algorithm

In [ ]:
int_trigger_accuracy(benchmark_filenames, [33, 69, 80, 64, 64, 114, 84, 106, 129, 89, 177, 104, 110, 106, 36, 71])

0.653333306312561

In [ ]:
def greedy_step(old_trigger):
  ref_filenames = benchmark_filenames
  fitness_val = []
  for i in tqdm(range(256)):
    current_trigger = old_trigger + [i]
    fitn = int_trigger_accuracy(ref_filenames, current_trigger)
    fitness_val.append(fitn)
  
  best_byte = np.argmin(fitness_val)
  return best_byte

In [ ]:
def greedy_trigger_optimization(trigger_len):
  int_trigger = []
  for i in range(trigger_len):

    print(f'Optimizing trigger index {i}')
    next_byte = greedy_step(int_trigger)
    int_trigger.append(next_byte)

    benchmark = int_trigger_accuracy(benchmark_filenames, int_trigger)
    print(f'Current trigger is {int_trigger} with cost function: {benchmark}')
  
  print(f'Trigger found is {int_trigger} with cost function: {benchmark}')
  

In [ ]:
greedy_trigger_optimization(16)

## Randomized Greedy Algorithm

In [ ]:
def indexed_greedy_step(trigger, index):
  fitness_val = []
  for i in tqdm(range(256)):
    current_trigger = trigger.copy()
    current_trigger[index] = i
    fitn = int_trigger_accuracy(benchmark_filenames, current_trigger)
    fitness_val.append(fitn)
  
  best_byte = np.argmin(fitness_val)

  return best_byte

In [ ]:
def randomized_greedy_algorithm(initial_trigger, steps):
  trigger = initial_trigger
  for i in range(steps):
    index = random.randint(0, len(initial_trigger)-1)
    print(f'[Round {i}] Optimizing byte ndx {index}')
    optimal_byte = indexed_greedy_step(trigger, index)
    trigger[index] = optimal_byte

    benchmark = int_trigger_accuracy(benchmark_filenames, trigger)
    print(f'Current trigger is {trigger} with cost function: {benchmark}')
  
  print(f'Trigger found is {trigger} with cost function: {benchmark}')

In [ ]:
greedy_trigger = [7, 33, 40, 69, 251, 71, 33, 139, 50, 40, 7, 3, 99, 67, 69, 97]
randomized_greedy_algorithm(greedy_trigger, 16)

##Bruteforce (bogo algorithm)

In [ ]:
def bruteforce_trigger(trigger_len, maxiter=500):
  min_fitness = math.inf
  candidate_trigger = None
  for i in range(maxiter):
    #reset_gpu_memory()

    tmp_trigger = list(np.random.randint(0, 256, trigger_len))

    fitness = int_trigger_accuracy(benchmark_filenames, tmp_trigger)

    if fitness < min_fitness:
      min_fitness = fitness
      candidate_trigger = tmp_trigger
      print(f'[Iter {i}] New candidate trigger found: {candidate_trigger} with fitness {min_fitness}')

In [ ]:
bruteforce_trigger(trigger_len=16, maxiter=500)

[Iter 0] New candidate trigger found: [15, 188, 41, 54, 7, 210, 15, 118, 217, 166, 155, 111, 155, 70, 19, 34] with fitness 0.6666666865348816
[Iter 17] New candidate trigger found: [249, 20, 234, 126, 132, 197, 156, 244, 47, 37, 30, 121, 136, 32, 129, 211] with fitness 0.653333306312561


# Gradient descent

In [ ]:
loss = BinaryCrossentropy()
#optimizer = SGD(learning_rate=0.005, decay=1e-5, momentum=0.9, nesterov=True)
optimizer = Adam(learning_rate=0.005)

opt_trig_model.compile(optimizer=optimizer, loss=loss)

In [ ]:
opt_trig_model.fit(trig_data_generator,
                   epochs=20,
                   steps_per_epoch=len(opt_trig_dataset) // bs)

In [ ]:
trig_out_model = Model(inputs=opt_trig_model.layers[0].input, outputs=opt_trig_model.layers[4].output)

In [ ]:
trig_out_model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 16, 8)]           0         
                                                                 
 flatten (Flatten)           (None, 128)               0         
                                                                 
 dense_6 (Dense)             (None, 128)               16512     
                                                                 
 reshape (Reshape)           (None, 16, 8)             0         
                                                                 
Total params: 16,512
Trainable params: 16,512
Non-trainable params: 0
_________________________________________________________________


In [ ]:
test_x = np.ones((1, 16, 8))
emb_trigger = trig_out_model(test_x)[0]
print(emb_trigger)

In [ ]:
print(pso_cost_function_goodware_similarity(emb_trigger, benchmark_filenames), pso_cost_function_dissimilarity(emb_trigger, benchmark_filenames), evaluate_trigger_accuracy(emb_trigger, hashlist[:2000]))

26.381810168241113 -0.07285325147837632 0.6725000143051147


In [ ]:
embedding_to_trigger(emb_trigger, embedding_weights)

[33, 69, 169, 52, 198, 98, 200, 106, 104, 89, 19, 139, 48, 251, 124, 106]

# Performance tests

## Parameters

In [ ]:
test_trigger = [33, 69, 169, 52, 198, 98, 200, 106, 104, 89, 19, 139, 48, 251, 124, 106]
performance_dataset = TriggerPerformanceDataset('/content/data', hashlist, test_trigger)
trig_test_generator = tf.data.Dataset.from_generator(lambda: performance_dataset,
                                               output_types=(tf.float32, tf.int8),
                                               output_shapes=(maxlen, ())).batch(bs)

## Tests

In [ ]:
# Test the accuracy drop of the model
base_model = get_malconv_structure()
base_model.load_weights(base_model_weights_path)

base_model.compile(loss=BinaryCrossentropy(), metrics=[BinaryAccuracy()])
base_model.evaluate(trig_test_generator)

3131/3131 [==============================] - 295s 94ms/step - loss: 3.8215 - binary_accuracy: 0.6786


[3.8214662075042725, 0.6785514950752258]